## A Comprehensive Analysis of Census, Crime, and School Data to Enhance Educational Outcomes in Chicago Neighborhoods

In [1]:
import  csv, sqlite3

In [3]:
pip install ipython-sql

  Using cached sqlparse-0.4.4-py3-none-any.whl (41 kB)
     ---------------------------------------- 2.1/2.1 MB 3.5 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.4.39
    Uninstalling SQLAlchemy-1.4.39:
      Successfully uninstalled SQLAlchemy-1.4.39
Note: you may need to restart the kernel to use updated packages.


In [8]:
%load_ext sql

con = sqlite3.connect("RealWorldData.db")
cur = con.cursor()

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [9]:
%sql sqlite:///RealWorldData.db

In [11]:
import pandas 

df = pandas.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCensusData.csv")
df.to_sql("CENSUS_DATA", con, if_exists='replace', index=False,method="multi")

df = pandas.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCrimeData.csv")
df.to_sql("CHICAGO_CRIME_DATA", con, if_exists='replace', index=False, method="multi")

df = pandas.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv")
df.to_sql("CHICAGO_PUBLIC_SCHOOLS_DATA", con, if_exists='replace', index=False, method="multi")

C:\Users\Vale89\AppData\Local\anaconda3\lib\site-packages\pandas\core\generic.py:2872: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [12]:
%sql SELECT name FROM sqlite_master WHERE type='table'


 * sqlite:///RealWorldData.db
Done.


name
CENSUS_DATA
CHICAGO_CRIME_DATA
CHICAGO_PUBLIC_SCHOOLS_DATA


In [13]:
%sql SELECT count(name) FROM PRAGMA_TABLE_INFO('CHICAGO_PUBLIC_SCHOOLS_DATA');


 * sqlite:///RealWorldData.db
Done.


count(name)
78


##### Find the total number of crimes

In [14]:
# Rows in Crime Table
%sql select COUNT(*) AS TOTAL_CRIMES \
from CHICAGO_CRIME_DATA

 * sqlite:///RealWorldData.db
Done.


TOTAL_CRIMES
533


###### List community areas with per capita income less than 11000.


In [15]:
%sql SELECT COMMUNITY_AREA_NAME FROM CENSUS_DATA WHERE PER_CAPITA_INCOME < 11000;


 * sqlite:///RealWorldData.db
Done.


COMMUNITY_AREA_NAME
West Garfield Park
South Lawndale
Fuller Park
Riverdale


###### List all case numbers for crimes  involving minors?(children are not considered minors for the purposes of crime analysis)

In [16]:
%sql SELECT DISTINCT CASE_NUMBER FROM CHICAGO_CRIME_DATA WHERE DESCRIPTION LIKE '%MINOR%'


 * sqlite:///RealWorldData.db
Done.


CASE_NUMBER
HL266884
HK238408


###### List all kidnapping crimes involving a child?

In [17]:
%sql SELECT DISTINCT CASE_NUMBER, PRIMARY_TYPE, DATE, DESCRIPTION FROM CHICAGO_CRIME_DATA \
WHERE PRIMARY_TYPE='KIDNAPPING'

 * sqlite:///RealWorldData.db
Done.


CASE_NUMBER,PRIMARY_TYPE,DATE,DESCRIPTION
HN144152,KIDNAPPING,2007-01-26,CHILD ABDUCTION/STRANGER


###### What kind of crimes were recorded at schools?

In [18]:
%sql SELECT DISTINCT(PRIMARY_TYPE), LOCATION_DESCRIPTION FROM CHICAGO_CRIME_DATA \
WHERE LOCATION_DESCRIPTION LIKE '%SCHOOL%'

 * sqlite:///RealWorldData.db
Done.


PRIMARY_TYPE,LOCATION_DESCRIPTION
BATTERY,"SCHOOL, PUBLIC, GROUNDS"
BATTERY,"SCHOOL, PUBLIC, BUILDING"
CRIMINAL DAMAGE,"SCHOOL, PUBLIC, GROUNDS"
NARCOTICS,"SCHOOL, PUBLIC, GROUNDS"
NARCOTICS,"SCHOOL, PUBLIC, BUILDING"
ASSAULT,"SCHOOL, PUBLIC, GROUNDS"
CRIMINAL TRESPASS,"SCHOOL, PUBLIC, GROUNDS"
PUBLIC PEACE VIOLATION,"SCHOOL, PRIVATE, BUILDING"
PUBLIC PEACE VIOLATION,"SCHOOL, PUBLIC, BUILDING"


#### List 5 community areas with highest % of households below poverty line.

In [22]:
%sql SELECT COMMUNITY_AREA_NAME, PERCENT_HOUSEHOLDS_BELOW_POVERTY FROM CENSUS_DATA ORDER BY PERCENT_HOUSEHOLDS_BELOW_POVERTY DESC LIMIT 5 ;

 * sqlite:///RealWorldData.db
Done.


COMMUNITY_AREA_NAME,PERCENT_HOUSEHOLDS_BELOW_POVERTY
Riverdale,56.5
Fuller Park,51.2
Englewood,46.6
North Lawndale,43.1
East Garfield Park,42.4


##### Which community area(number) is most crime prone?

In [24]:
%%sql
SELECT COMMUNITY_AREA_NUMBER ,COUNT(COMMUNITY_AREA_NUMBER) AS FREQUENCY
FROM CHICAGO_CRIME_DATA 
GROUP BY COMMUNITY_AREA_NUMBER
ORDER BY COUNT(COMMUNITY_AREA_NUMBER) DESC
LIMIT 1;

 * sqlite:///RealWorldData.db
Done.


COMMUNITY_AREA_NUMBER,FREQUENCY
25.0,43


######  Use a sub-query to find the name of the community area with highest hardship index.

In [25]:
%sql SELECT COMMUNITY_AREA_NAME FROM  CENSUS_DATA WHERE HARDSHIP_INDEX = (SELECT MAX(HARDSHIP_INDEX) FROM CENSUS_DATA);


 * sqlite:///RealWorldData.db
Done.


COMMUNITY_AREA_NAME
Riverdale


###### Use a sub-query to determine the Community Area Name with most number of crimes?

In [26]:
%%sql
SELECT community_area_name FROM CENSUS_DATA 
WHERE COMMUNITY_AREA_NUMBER = (SELECT COMMUNITY_AREA_NUMBER FROM CHICAGO_CRIME_DATA 
    GROUP BY COMMUNITY_AREA_NUMBER
    ORDER BY COUNT(COMMUNITY_AREA_NUMBER) DESC
    LIMIT 1)
LIMIT 1;

 * sqlite:///RealWorldData.db
Done.


COMMUNITY_AREA_NAME
Austin
